# 1. Import thư viện

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

# 2. Nhập dữ liệu
**Tập dữ liệu đầy đủ**: Bao gồm 26.000.000 đánh giá và 750.000 thẻ được áp dụng cho 45.000 bộ phim bởi 270.000 người dùng. Bao gồm dữ liệu genome thẻ với 12 triệu điểm liên quan trên 1.100 thẻ.

**Tập dữ liệu nhỏ**: Bao gồm 100.000 đánh giá và 1.300 thẻ được áp dụng cho 9.000 bộ phim bởi 700 người dùng.

Chúng ta sẽ xây dựng hệ thống gợi ý đơn giản của mình sử dụng các bộ phim từ tập dữ liệu đầy đủ.

In [2]:
credits = pd.read_csv('input_data/credits.csv')
keywords = pd.read_csv('input_data/keywords.csv')
links_small = pd.read_csv('input_data/links_small.csv')
md = pd.read_csv('input_data/movies_metadata.csv')
ratings = pd.read_csv('input_data/ratings_small.csv')

# 3. Phân tích dữ liệu
## Tệp dữ liệu credits.csv

In [3]:
credits.head()
#credits.iloc[0:3]
# credits['cast'].iloc[0:3]
# credits.iloc[:,0:2]

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
credits.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [5]:
credits.shape

(45476, 3)

In [6]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


**Gồm**: 45476 hàng non-null, 3 cột 'cast', 'crew', 'id'

* **cast**: gồm thông tin về diễn viên chính, dưới dạng json
* **crew**: gồm thông tin về đoàn phim, dưới dạng json
* **id**: id của phim trong metadata

## Tệp dữ liệu keywords.csv

In [7]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [8]:
keywords.columns

Index(['id', 'keywords'], dtype='object')

In [9]:
keywords.shape

(46419, 2)

In [10]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


**Gồm**: 46419  hàng non-null, 3 cột 'id', 'keywords'

* **id**: id của phim trong metadata
* **keywords**: gồm thẻ(tag) và keywords cho bộ phim.


## Tệp dữ liệu movies_metadata.csv

In [11]:
md.iloc[0:3].transpose()

,0,1,2
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
budget,30000000,65000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
homepage,http://toystory.disney.com/toy-story,NaN,NaN
id,862,8844,15602
imdb_id,tt0114709,tt0113497,tt0113228
original_language,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...


In [12]:
md.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [13]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

**Các Features:**

* **adult:** Đánh giá bộ phim có phù hợp cho trẻ em hay không (dưới 18 tuổi)
* **belongs_to_collection:** Chứa thông tin về bộ (series) của bộ phim, dưới dạng json
* **budget:** Kinh phí của bộ phim, tính bằng đô
* **genres:** Chứa thông tin về thể loại của bộ phim, dưới dạng json
* **homepage:** Trang chính thức của bộ phim
* **id:** ID của bộ phim
* **imdb_id:** ID của phim trong csdl IMBD     
* **original_language:** Ngôn ngữ của phim
* **original_title:** Tựa đề gốc của phim
* **overview:** Mô tả ngắn gọn của phim
* **popularity:** Độ phổ biến, đánh giá của TMDB
* **poster_path:** URL của ảnh bìa phim
* **production_companies:** Chứa thông tin về các công ty tham gia sản xuất, dưới dạng json
* **production_countries:** Chứa thông tin về quốc gia mà bộ phim được quay, dưới dạng json
* **release_date:** Ngày ra mắt được công bố của phim
* **revenue:** Tổng doanh thu của phim
* **runtime:** Thời lượng của phim, tính bằng phút
* **spoken_languages:** Chứa thôn tin về các ngôn ngữ được nói trong phim, dưới dạng json
* **status:** Trạng thái của phim (Released, To Be Released, Announced, etc)
* **tagline:** Tagline của bộ phim
* **title:** Tựa đề chính thức của phim
* **video:** Bộ phim có video giới thiệu trên TMDB hay không
* **vote_average:** Đánh giá trung bình của bộ phim.
* **vote_count:** Số lượng đánh giá của người dùng, trên TMDB

In [14]:
md["spoken_languages"].values

array(["[{'iso_639_1': 'en', 'name': 'English'}]",
       "[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",
       "[{'iso_639_1': 'en', 'name': 'English'}]", ...,
       "[{'iso_639_1': 'en', 'name': 'English'}]", '[]',
       "[{'iso_639_1': 'en', 'name': 'English'}]"], dtype=object)

In [15]:
md.shape

(45466, 24)

## Tệp dữ liệu ratings.csv

In [16]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [17]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

* **userId:** ID của người dùng
* **movieId:** ID của phim trong TMDB
* **rating:** Số điểm đánh giá của một người dùng cụ thể
* **timestamp:** Thời điểm thực hiện đánh giá

In [18]:
ratings.shape

(100004, 4)

In [19]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


# 4. Tiền xử lý dữ liệu

* Đến đâu xử lý đến đó

# 5. Xây dựng hệ thống gợi ý (Recommendation System)
# 5.1. Hệ thống gợi ý đơn giản
**Cách thức tiếp cận:**

* Đối với 1 hệ thống đơn giản sẽ đưa ra __gợi ý đơn thuần, tổng quát__ tới tất cả người dùng __dựa trên độ phổ biến của phim và thể loại của phim__. 
* * **Input:** Chọn 1 bộ phim bất kỳ
* * **Xử lý:** Dựa trên độ phổ biến và thể loại của phim
* * **Output:** Cho ra những bộ phim tương tự
* * **Nhược điểm:** Với cùng 1 Input, luôn luôn cho ra kết quả giống nhau, và không sử dụng model, mà thay vào đó, sử dụng __cosine_similarity__. Về cơ bản, __những bộ phim phổ biến hơn và được đánh giá cao hơn sẽ có xác suất cao hơn được khán giả phổ thông yêu thích, và nó không có tính cá nhân hoá cho từng người dùng.__ 

**Thực tế:**

* Việc ứng dụng model này là ở mức vô dùng đơn giản. 
* Chúng ta chỉ đơn giản __sắp xếp các bộ phim dựa trên đánh giá và độ phổ biến__ sau đó đưa ra top những phim đó trong danh sách đã có. 
* Ngoài ra, chúng ta cũng __thêm vào tham số về thể loại để đưa thể loại vào cân nhắc top các phim.__

Chúng ta sẽ xây dựng 1 bảng top 250 bộ phim cho những thể loại cụ thể.

In [20]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval)
md['genres'] = md['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45466, dtype: object

In [22]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)
gen_md.head(3).transpose()

,0,0,0
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ...","{'id': 10194, 'name': 'Toy Story Collection', ..."
budget,30000000,30000000,30000000
homepage,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story,http://toystory.disney.com/toy-story
id,862,862,862
imdb_id,tt0114709,tt0114709,tt0114709
original_language,en,en,en
original_title,Toy Story,Toy Story,Toy Story
overview,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ..."
popularity,21.946943,21.946943,21.946943


In [28]:
gen_md['genre'].value_counts()


genre
Drama                                    20265
Comedy                                   13182
Thriller                                  7624
Romance                                   6735
Action                                    6596
Horror                                    4673
Crime                                     4307
Documentary                               3932
Adventure                                 3496
Science Fiction                           3049
Family                                    2770
Mystery                                   2467
Fantasy                                   2313
Animation                                 1935
Foreign                                   1622
Music                                     1598
History                                   1398
War                                       1323
Western                                   1042
TV Movie                                   767
Carousel Productions                         1
Vision 

In [29]:
links_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB
